# Generating Synthetic PV Power Time Series
Evaluation / Verification

In [1]:
import os
import yaml
import numpy as np
import pandas as pd
import pvlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
def load_config(config_path):
    with open(config_path, "r") as file:
        return yaml.safe_load(file)

In [3]:
config_path = "config.yaml"
config = load_config(config_path)

dir = config['data']['final_data']
params = config['synth']
adj_params = config['adjustable_pv_params']

plot_names = ['Direct', 'Diffuse']

In [4]:
files = os.listdir(dir)
file = files[0]

df = pd.read_csv(os.path.join(dir, file))
df['timestamp'] = pd.to_datetime(df['timestamp'])
#df['timestamp'] = df['timestamp'].dt.tz_localize("UTC").dt.tz_convert("Europe/Berlin")
df.set_index('timestamp', inplace=True)

df.drop(['  QN_x', '  QN_y', '  QN'], axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 79200 entries, 2023-05-26 00:00:00 to 2024-11-25 23:50:00
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   STATIONS_ID    79200 non-null  int64  
 1   PP_10          79200 non-null  float64
 2   TT_10          79200 non-null  float64
 3   TM5_10         79200 non-null  float64
 4   RF_10          79200 non-null  float64
 5   TD_10          79200 non-null  float64
 6   FF_10          79200 non-null  float64
 7   DD_10          79200 non-null  int64  
 8   DS_10          79200 non-null  float64
 9   GS_10          79200 non-null  float64
 10  SD_10          79200 non-null  float64
 11  LS_10          79200 non-null  int64  
 12  Stationshoehe  79200 non-null  float64
 13  geoBreite      79200 non-null  float64
 14  geoLaenge      79200 non-null  float64
dtypes: float64(12), int64(3)
memory usage: 9.7 MB


In [12]:
def get_features(data: pd.DataFrame, 
                 params: dict,
                 adj_params: dict,
                 az = 180,
                 tilt = 30
                ):
    # calculate pressure
    #pressure = pvlib.atmosphere.alt2pres(elevation)
    dhi = data[params['dhi']['param']]
    ghi = data[params['ghi']['param']]
    pressure = data[params['pressure']['param']]
    temperature = data[params['temperature']['param']]
    v_wind = data[params['v_wind']['param']]
    latitude = data[params['latitude']['param']]
    longitude = data[params['longitude']['param']]
    elevation = data[params['elevation']['param']]
    
    surface_tilt = tilt #adj_params['surface_tilt']
    surface_azimuth = az #adj_params['surface_azimuth']
    albedo = adj_params['albedo']
    
    # get solar position
    solpos = pvlib.solarposition.get_solarposition(
        time=data.index,
        latitude=latitude,
        longitude=longitude,
        altitude=elevation,
        temperature=temperature,
        pressure=pressure,
    )
    solar_zenith = solpos['zenith']
    solar_azimuth = solpos['azimuth']
    
    # GHI and DHI in W/m^2 --> J / cm^2 = J / 0,0001 m^2 = 10000 J / m^2 --> Dividing by 600 seconds (DWD is giving GHI as sum of 10 minutes))
    dhi = data[params['dhi']['param']] * 1e4 / 600
    ghi = data[params['ghi']['param']] * 1e4 / 600
    
    # get dni from ghi, dni and zenith
    dni = pvlib.irradiance.dni(ghi=ghi,
                               dhi=dhi,
                               zenith=solar_zenith)
    
    data['dni'] = dni
    data['dhi'] = dhi
    
    # get total irradiance
    total_irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=surface_tilt,
        surface_azimuth=surface_azimuth,
        solar_zenith=solar_zenith,
        solar_azimuth=solar_azimuth,
        dni=dni,
        ghi=ghi,
        dhi=dhi,
        dni_extra=pvlib.irradiance.get_extra_radiation(data.index),
        albedo=albedo,
        model='haydavies',
    )
    
    cell_temperature = pvlib.temperature.faiman(total_irradiance['poa_global'], 
                                                temperature,
                                                v_wind, 
                                                u0=25.0, 
                                                u1=6.84)
    
    return total_irradiance, cell_temperature


def generate_pv_power(total_irradiance: pd.Series,
                      cell_temperature: pd.Series,
                      adj_params: dict                      
                      ) -> pd.Series:
    
    installed_power = adj_params['installed_power']
    gamma_pdc = adj_params['gamma_pdc']
    
    power_dc = pvlib.pvsystem.pvwatts_dc(total_irradiance, 
                                         cell_temperature, 
                                         installed_power,
                                         gamma_pdc=gamma_pdc, 
                                         temp_ref=25.0)
    
    gross_power = pvlib.inverter.pvwatts(power_dc, 
                                         installed_power, 
                                         eta_inv_nom=0.96, 
                                         eta_inv_ref=0.9637)
    
    losses = pvlib.pvsystem.pvwatts_losses(soiling=2,
                                           shading=3,
                                           snow=0,
                                           mismatch=2,
                                           wiring=2,
                                           connections=0.5,
                                           lid=1.5,
                                           nameplate_rating=1,
                                           age=0,
                                           availability=3)
    
    net_power = gross_power * (1-losses)
    
    return net_power 
    
    
def plot_power_and_features(day: str, 
                            data: pd.DataFrame,
                            plot_names: list,
                            power: list,
                            values: list,
                            of_interest='AZ',
                            plot_irradiance=True,
                            synchronize_axes=True,
                            save_fig=False
                            ): 
    
    az_mapping = {0: 'N',
                  90: 'E',
                  180: 'S',
                  270: 'W'}
    
    day = pd.Timestamp(day)
    index_0 = power[0].index.get_loc(day)
    index_1 = power[0].index.get_loc(day + pd.Timedelta(days=1))
    date = str(power[0].index[index_0:index_1][0].date())

    fig, ax1 = plt.subplots(figsize=(10, 6))
    fontsize = 14
    lines = []
    title_prefix = ''
    title_suffix = ''
    
    colors = ['#4e79a7', '#f28e2b', '#76b7b2', '#e15759'] 
    rad_colors = ['#000000', '#b07aa1']
    
    for p, value in enumerate(values):
        tilt = 30 if of_interest == 'AZ' else value
        az = 180 if of_interest == 'Tilt' else value
        label_suffix = f'AZ: {az_mapping[az]} Tilt: {tilt}°'
        # plot power
        line, = ax1.plot(
        power[p][index_0:index_1],
        label=f"{label_suffix}",
        color=colors[p],
        linewidth=2.0
        )
        lines.append(line)

    # configure secondary y-axis
    ax1.set_xlabel("Time", fontsize=fontsize)
    ax1.set_ylabel("Power Output (W)", fontsize=fontsize)
    ax1.tick_params(axis='y', labelsize=fontsize)
    ax1.tick_params(axis='x', labelsize=fontsize-2)
    
    if plot_irradiance:
        irradiances = [data['dni'], data['dhi']]
        ax2 = ax1.twinx()
        # plot irradiance components
        for color, name, series in zip(rad_colors, plot_names, irradiances):
            line, = ax2.plot(
                series[index_0:index_1],
                label=name,
                color=color,
                linestyle='--',
                linewidth=2.0
            )
            lines.append(line)

        # configure primary y-axis
        ax2.set_ylabel("Energy flux density (W/m$^2$)", fontsize=fontsize)
        ax2.tick_params(axis='y', labelsize=fontsize)
        title_prefix = 'Irradiance in Watt/m$^2$ and '

    # Format x-axis to show only hours (HH)
    ax1.xaxis.set_major_locator(mdates.HourLocator(interval=1)) 
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H'))
    ticks = ax1.get_xticks()
    ax1.set_xticks(ticks[1:-1])


    # Synchronize y-axes
    if synchronize_axes:
        title_suffix = '(synched axes)'
        all_ghi_min = min([series[index_0:index_1].min() for series in irradiances])
        all_ghi_max = max([series[index_0:index_1].max() for series in irradiances])
        y_min = min(all_ghi_min, power[0][index_0:index_1].min())
        y_max = max(all_ghi_max, power[0][index_0:index_1].max())
        ax1.set_ylim(y_min, y_max)
        ax2.set_ylim(y_min, y_max)

    # legend
    #lines.append(lines.pop(0))
    labels = [line.get_label() for line in lines]
    ax1.legend(lines, labels, loc="upper left", fontsize=fontsize)
    
    plt.title(f"{title_prefix}Power in Watt on {date} {title_suffix}", fontsize=fontsize)
    fig.tight_layout()
    #plt.grid(True)
    if synchronize_axes:
        suffix = '_sync_ax'
    else:
        suffix = ''
    if save_fig:
        save_path = 'figs/eval'
        os.makedirs(save_path, exist_ok=True)
        save_file = os.path.join(save_path, f'{date}_{of_interest}{suffix}.png')
        plt.savefig(save_file, dpi=300)
        plt.close()        
    else:
        plt.show()
        
def get_time_series(df: pd.DataFrame,
                    params: dict,
                    adj_params: dict,
                    values: list,
                    of_interest = 'AZ'):
    irradiances = []
    powers = []
    for value in values:
        total_irradiance, cell_temperature = get_features(data=df,
                                                        params=params,
                                                        adj_params=adj_params,
                                                        az=value if of_interest == 'AZ' else 180,
                                                        tilt=value if of_interest == 'Tilt' else 30)
        total = total_irradiance['poa_global']
        direct = total_irradiance['poa_direct']
        diffuse = total_irradiance['poa_diffuse']
        #sky_dhi = total_irradiance['poa_sky_diffuse']
        #ground_dhi = total_irradiance['poa_ground_diffuse']
        features = [direct, diffuse]
        power = generate_pv_power(total_irradiance=total,
                                cell_temperature=cell_temperature,
                                adj_params=adj_params)

        irradiances.append(features)
        powers.append(power)
        
    return irradiances, powers

In [10]:
azs = [0, 90, 180, 270]
tilts = [0, 30, 60, 90]

az_irradiances, az_powers = get_time_series(df=df, 
                                            params=params, 
                                            adj_params=adj_params, 
                                            values=azs,
                                            of_interest='AZ')

tilt_irradiances, tilt_powers = get_time_series(df=df, 
                                                params=params, 
                                                adj_params=adj_params, 
                                                values=tilts,
                                                of_interest='Tilt')

In [13]:
day = '2023-09-04'

# plot_power_and_features(day=day,
#                         data=df,
#                         plot_names=plot_names,
#                         power=az_powers,
#                         values=azs,
#                         of_interest='AZ',
#                         plot_irradiance=True,
#                         synchronize_axes=False,
#                         save_fig=True)

plot_power_and_features(day=day,
                        data=df,
                        plot_names=plot_names,
                        power=tilt_powers,
                        values=tilts,
                        of_interest='Tilt',
                        plot_irradiance=True,
                        synchronize_axes=False,
                        save_fig=True)

In [102]:
days = ['2023-06-04', '2023-07-07', '2023-09-04',
       '2023-08-27', '2023-08-29', '2023-10-17']

for day in days:
    plot_power_and_features(day=day,
                            data=df,
                            plot_names=plot_names,
                            power=az_powers,
                            values=azs,
                            of_interest='AZ',
                            plot_irradiance=True,
                            synchronize_axes=False,
                            save_fig=True)

    plot_power_and_features(day=day,
                            data=df,
                            plot_names=plot_names,
                            power=tilt_powers,
                            values=tilts,
                            of_interest='Tilt',
                            plot_irradiance=True,
                            synchronize_axes=False,
                            save_fig=True)